In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 188.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 142.9 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.24.0
    Uninstalling wandb-0.24.0:
      Successfully uninstalled wandb-0.24.0


In [ ]:
# Install required packages
!pip install --upgrade --quiet \
    bitsandbytes \
    datasets \
    evaluate \
    peft \
    tensorboard \
    transformers \
    trl \
    accelerate \
    wandb

print("All packages installed successfully")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 174.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 51.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires tensorboard~=2.19.0, but you have tensorboard 2.20.0 which is incompatible.
All packages installed successfully


In [ ]:
import os
import json

# Reproducibility seed
GLOBAL_SEED = 42
os.environ["PYTHONHASHSEED"] = str(GLOBAL_SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_SILENT"] = "True"

# API Keys
HF_TOKEN = ""
WANDB_API_KEY = ""

# Kaggle Legacy Credentials
KAGGLE_USERNAME = ""
KAGGLE_KEY = ""

# Set environment variables BEFORE any imports that use them
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY

# Write kaggle.json for libraries that check the file
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
kaggle_json = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_json, f)
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

# Authenticate HuggingFace
from huggingface_hub import login
login(token=HF_TOKEN, add_to_git_credential=False)
print("HuggingFace authentication successful")

# Authenticate Weights & Biases
import wandb
wandb.login(key=WANDB_API_KEY, relogin=True)
print("Weights & Biases authentication successful")

print("Kaggle authentication successful")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


HuggingFace authentication successful


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Weights & Biases authentication successful
Kaggle authentication successful


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTConfig, SFTTrainer
import wandb

# Configuration
GLOBAL_SEED = 42
MODEL_ID = "google/medgemma-1.5-4b-it"
DATASET_ID = "Nadhari/MedToolCalling"
OUTPUT_DIR = "medgemma-4b-tool-calling"
WANDB_PROJECT = "MedGemmaAgent"
WANDB_RUN_NAME = "Sara-1.5-4B-it"

# Training config
MAX_SEQ_LENGTH = 16384  # Covers max sample of ~15K tokens
NUM_TRAIN_SAMPLES = 264
NUM_EVAL_SAMPLES = 20

# Set seeds for reproducibility
torch.manual_seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA H100 80GB HBM3
GPU Memory: 85.2 GB


In [ ]:
# Initialize Weights & Biases
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_RUN_NAME,
    config={
        "model_id": MODEL_ID,
        "dataset_id": DATASET_ID,
        "max_seq_length": MAX_SEQ_LENGTH,
        "num_train_samples": NUM_TRAIN_SAMPLES,
        "num_eval_samples": NUM_EVAL_SAMPLES,
        "seed": GLOBAL_SEED,
    }
)

print(f"WandB initialized: {WANDB_PROJECT}/{WANDB_RUN_NAME}")

WandB initialized: MedGemmaAgent/Sara-1.5-4B-it


In [ ]:
# Load the MedToolCalling dataset
print("Loading dataset...")
raw_dataset = load_dataset(DATASET_ID)
print(f"Dataset loaded: {raw_dataset}")
print(f"Total samples: {len(raw_dataset['train'])}")

# Preview a sample
sample = raw_dataset['train'][0]
print(f"\nSample keys: {sample.keys()}")
print(f"Sample ID: {sample['id']}")
print(f"Num turns: {sample['num_turns']}")
print(f"Conversation length: {len(sample['conversation'])}")

Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.96k [00:00<?, ?B/s]

dataset.jsonl:   0%|          | 0.00/6.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/284 [00:00<?, ? examples/s]

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'context', 'conversation', 'num_turns', 'final_answer'],
        num_rows: 284
    })
})
Total samples: 284

Sample keys: dict_keys(['id', 'instruction', 'context', 'conversation', 'num_turns', 'final_answer'])
Sample ID: task1_1
Num turns: 2
Conversation length: 4


In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Set padding token if not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")
print(f"Vocab size: {tokenizer.vocab_size:,}")
print(f"Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")

# Check special tokens for Gemma
print(f"\nSpecial tokens map: {tokenizer.special_tokens_map}")

Loading tokenizer...


config.json:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Tokenizer loaded: GemmaTokenizer
Vocab size: 262,144
Pad token: <pad> (ID: 0)
EOS token: <eos> (ID: 1)
BOS token: <bos> (ID: 2)

Special tokens map: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}


In [ ]:
# Analyze token distribution across the dataset
print("Analyzing token distribution...")

def analyze_sample(sample):
    """Analyze tokens in a single sample"""
    conversation = sample['conversation']

    # Build full conversation text in Gemma format
    messages = []
    for turn in conversation:
        role = "user" if turn['role'] == "user" else "model"
        messages.append({"role": role, "content": turn['content']})

    # Tokenize with chat template
    full_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    full_tokens = tokenizer.encode(full_text, add_special_tokens=False)

    # Count agent (model) tokens only
    agent_text = ""
    for turn in conversation:
        if turn['role'] == 'agent':
            agent_text += turn['content'] + "\n"
    agent_tokens = tokenizer.encode(agent_text, add_special_tokens=False)

    return {
        'total_tokens': len(full_tokens),
        'agent_tokens': len(agent_tokens),
        'input_tokens': len(full_tokens) - len(agent_tokens),
        'task_type': sample['id'].split('_')[0]
    }

# Analyze all samples
analysis_results = [analyze_sample(s) for s in raw_dataset['train']]
df_analysis = pd.DataFrame(analysis_results)

print("\n" + "=" * 60)
print("TOKEN STATISTICS")
print("=" * 60)
print(f"\n{'Metric':<20} {'Min':>10} {'Max':>10} {'Mean':>12} {'Median':>12}")
print("-" * 66)
for col in ['total_tokens', 'agent_tokens', 'input_tokens']:
    data = df_analysis[col]
    print(f"{col:<20} {data.min():>10} {data.max():>10} {data.mean():>12.1f} {data.median():>12.1f}")

print(f"\nTotal tokens in dataset: {df_analysis['total_tokens'].sum():,}")
print(f"Agent tokens (training target): {df_analysis['agent_tokens'].sum():,}")
print(f"Samples exceeding {MAX_SEQ_LENGTH} tokens: {(df_analysis['total_tokens'] > MAX_SEQ_LENGTH).sum()}")

# Log to wandb
wandb.log({
    "dataset/total_samples": len(df_analysis),
    "dataset/total_tokens": df_analysis['total_tokens'].sum(),
    "dataset/agent_tokens": df_analysis['agent_tokens'].sum(),
    "dataset/max_tokens": df_analysis['total_tokens'].max(),
    "dataset/mean_tokens": df_analysis['total_tokens'].mean(),
})

Analyzing token distribution...

TOKEN STATISTICS

Metric                      Min        Max         Mean       Median
------------------------------------------------------------------
total_tokens               3569      15141       5585.6       4212.0
agent_tokens                 32        561        119.7         66.0
input_tokens               3374      15108       5465.9       4159.5

Total tokens in dataset: 1,586,311
Agent tokens (training target): 33,982
Samples exceeding 16384 tokens: 0


In [ ]:
def format_conversation_for_training(sample):
    """
    Convert dataset conversation to Gemma chat format.
    Maps 'user' -> 'user', 'agent' -> 'model'
    """
    conversation = sample['conversation']

    messages = []
    for turn in conversation:
        # Map roles: user stays user, agent becomes model
        role = "user" if turn['role'] == "user" else "model"
        messages.append({
            "role": role,
            "content": turn['content']
        })

    return {"messages": messages}


# Apply formatting to all samples
print("Formatting dataset for training...")
formatted_dataset = raw_dataset['train'].map(
    format_conversation_for_training,
    remove_columns=['instruction', 'context', 'conversation', 'num_turns', 'final_answer'],
    desc="Formatting conversations"
)

print(f"Formatted dataset: {formatted_dataset}")
print(f"\nSample formatted entry:")
print(f"  ID: {formatted_dataset[0]['id']}")
print(f"  Num messages: {len(formatted_dataset[0]['messages'])}")
for i, msg in enumerate(formatted_dataset[0]['messages']):
    content_preview = msg['content'][:80].replace('\n', ' ')
    print(f"  [{i}] {msg['role']}: {content_preview}...")

Formatting dataset for training...


Formatting conversations:   0%|          | 0/284 [00:00<?, ? examples/s]

Formatted dataset: Dataset({
    features: ['id', 'messages'],
    num_rows: 284
})

Sample formatted entry:
  ID: task1_1
  Num messages: 4
  [0] user: You are an expert in using FHIR functions to assist medical professionals. You a...
  [1] model: GET http://localhost:8080/fhir/Patient?given=Peter&family=Stafford&birthdate=193...
  [2] user: Here is the response from the GET request: {   "resourceType": "Bundle",   "id":...
  [3] model: FINISH(["S6534835"])...


In [ ]:
# Shuffle and split the dataset
print("Splitting dataset...")

# Shuffle with seed for reproducibility
shuffled_dataset = formatted_dataset.shuffle(seed=GLOBAL_SEED)

# Split: 264 train, 20 eval
train_dataset = shuffled_dataset.select(range(NUM_TRAIN_SAMPLES))
eval_dataset = shuffled_dataset.select(range(NUM_TRAIN_SAMPLES, NUM_TRAIN_SAMPLES + NUM_EVAL_SAMPLES))

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

# Verify no overlap
train_ids = set(train_dataset['id'])
eval_ids = set(eval_dataset['id'])
assert len(train_ids.intersection(eval_ids)) == 0, "Train/eval overlap detected!"
print("No overlap between train and eval sets")

# Log split info
wandb.log({
    "dataset/train_samples": len(train_dataset),
    "dataset/eval_samples": len(eval_dataset),
})

Splitting dataset...
Train samples: 264
Eval samples: 20
No overlap between train and eval sets


In [ ]:
from typing import Any, List, Dict

def collate_fn(examples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
    """
    Custom collator that:
    1. Applies Gemma chat template
    2. Tokenizes and pads
    3. Creates token_type_ids (required by Gemma 3)
    4. Masks loss on non-assistant (non-model) tokens
    """
    texts = []

    for example in examples:
        text = tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    # Tokenize all texts with padding

    # Change this line in collate_fn:
    batch = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,  # Pad to longest in batch (dynamic)
        truncation=True,
        max_length=MAX_SEQ_LENGTH,  # Only truncate if exceeds, don't pad to this
        )

    # Create labels (copy of input_ids)
    labels = batch["input_ids"].clone()

    # Mask padding tokens
    labels[labels == tokenizer.pad_token_id] = -100

    # Create token_type_ids: 0 for user, 1 for model
    # This is REQUIRED for Gemma 3 during training
    token_type_ids = torch.zeros_like(batch["input_ids"])

    # Get token IDs for markers
    model_start_tokens = tokenizer.encode("<start_of_turn>model\n", add_special_tokens=False)
    user_start_tokens = tokenizer.encode("<start_of_turn>user\n", add_special_tokens=False)
    end_turn_tokens = tokenizer.encode("<end_of_turn>", add_special_tokens=False)

    for batch_idx in range(len(labels)):
        input_ids = batch["input_ids"][batch_idx].tolist()

        in_model_response = False
        i = 0
        while i < len(input_ids):
            # Check for start of model turn
            if (i + len(model_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(model_start_tokens)] == model_start_tokens):
                # Mark the start tokens as model type
                for j in range(len(model_start_tokens)):
                    token_type_ids[batch_idx, i + j] = 1
                    labels[batch_idx, i + j] = -100  # Don't predict turn markers
                i += len(model_start_tokens)
                in_model_response = True
                continue

            # Check for start of user turn
            if (i + len(user_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(user_start_tokens)] == user_start_tokens):
                for j in range(len(user_start_tokens)):
                    token_type_ids[batch_idx, i + j] = 0
                    labels[batch_idx, i + j] = -100
                i += len(user_start_tokens)
                in_model_response = False
                continue

            # Check for end of turn
            if (i + len(end_turn_tokens) <= len(input_ids) and
                input_ids[i:i+len(end_turn_tokens)] == end_turn_tokens):
                if in_model_response:
                    # Model should learn to produce end token
                    for j in range(len(end_turn_tokens)):
                        token_type_ids[batch_idx, i + j] = 1
                else:
                    for j in range(len(end_turn_tokens)):
                        token_type_ids[batch_idx, i + j] = 0
                        labels[batch_idx, i + j] = -100
                i += len(end_turn_tokens)
                in_model_response = False
                continue

            # Set token type and mask based on current state
            if in_model_response:
                token_type_ids[batch_idx, i] = 1
                # Keep label (model should predict this)
            else:
                token_type_ids[batch_idx, i] = 0
                labels[batch_idx, i] = -100  # Mask user tokens

            i += 1

    batch["labels"] = labels
    batch["token_type_ids"] = token_type_ids

    return batch


# Test the collator with one sample
print("Testing collator...")
test_batch = collate_fn([train_dataset[0]])
print(f"Input shape: {test_batch['input_ids'].shape}")
print(f"Labels shape: {test_batch['labels'].shape}")
print(f"Token type IDs shape: {test_batch['token_type_ids'].shape}")

# Verify token_type_ids has both 0s and 1s
print(f"Token type 0 (user) count: {(test_batch['token_type_ids'] == 0).sum().item()}")
print(f"Token type 1 (model) count: {(test_batch['token_type_ids'] == 1).sum().item()}")

# Count training tokens
non_pad_mask = test_batch['input_ids'] != tokenizer.pad_token_id
total_real_tokens = non_pad_mask.sum().item()
training_tokens = (test_batch['labels'] != -100).sum().item()

print(f"Total tokens (non-pad): {total_real_tokens}")
print(f"Training target tokens (unmasked): {training_tokens}")
print(f"Training signal ratio: {training_tokens / total_real_tokens * 100:.1f}%")

Testing collator...
Input shape: torch.Size([1, 5405])
Labels shape: torch.Size([1, 5405])
Token type IDs shape: torch.Size([1, 5405])
Token type 0 (user) count: 5359
Token type 1 (model) count: 46
Total tokens (non-pad): 5405
Training target tokens (unmasked): 40
Training signal ratio: 0.7%


In [ ]:
# Configure 4-bit quantization for efficient training
print("Loading model with 4-bit quantization...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.bfloat16,  # Changed from torch_dtype
    attn_implementation="flash_attention_2",  # Use Flash Attention 2 for H100
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

print(f"Model loaded: {model.__class__.__name__}")
print(f"Model dtype: {model.dtype}")
print(f"Device map: {getattr(model, 'hf_device_map', 'N/A')}")  # Safe access

# Print memory usage
if torch.cuda.is_available():
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU Memory reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

Loading model with 4-bit quantization...


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Model loaded: Gemma3ForConditionalGeneration
Model dtype: torch.float32
Device map: N/A
GPU Memory allocated: 4.60 GB
GPU Memory reserved: 10.76 GB


In [ ]:
# Configure LoRA for parameter-efficient fine-tuning
peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

# Log to wandb
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
wandb.log({
    "model/trainable_params": trainable_params,
    "model/total_params": total_params,
    "model/trainable_percent": trainable_params / total_params * 100,
})

trainable params: 32,788,480 || all params: 4,332,867,952 || trainable%: 0.7567


In [ ]:
# Training configuration optimized for H100 + small dataset
# Small dataset (264 samples) -> careful with epochs to avoid overfitting

# Calculate steps
batch_size = 1
grad_accum = 16
effective_batch_size = batch_size * grad_accum  # 16
steps_per_epoch = len(train_dataset) // effective_batch_size  # ~16 steps/epoch

# Use epochs as a balance (not too few, not too many for 264 samples)
num_epochs = 4

print(f"Training configuration:")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Batch size: {batch_size}")
print(f"  Gradient accumulation: {grad_accum}")
print(f"  Effective batch size: {effective_batch_size}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Num epochs: {num_epochs}")
print(f"  Total steps: {steps_per_epoch * num_epochs}")

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,

    # Epochs and steps
    num_train_epochs=num_epochs,
    max_steps=-1,  # Use num_train_epochs

    # Batch sizes (H100 can handle this)
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,

    # Optimizer settings
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    weight_decay=0.01,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    # Precision
    bf16=True,
    tf32=True,

    # Memory optimization
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},

    # Logging
    logging_steps=5,
    logging_first_step=True,
    report_to="wandb",

    # Evaluation
    eval_strategy="steps",
    eval_steps=8,

    # Saving
    save_strategy="steps",
    save_steps=16,
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Hub
    push_to_hub=False,  # Set True if you want to push to Hub
    hub_model_id=OUTPUT_DIR,

    # Dataset handling
    dataset_kwargs={"skip_prepare_dataset": True},
    remove_unused_columns=False,
    label_names=["labels"],

    # Sequence length
    max_length=MAX_SEQ_LENGTH,

    # Seed
    seed=GLOBAL_SEED,
    data_seed=GLOBAL_SEED,
)

print(f"\nTraining args configured")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Training configuration:
  Train samples: 264
  Batch size: 1
  Gradient accumulation: 16
  Effective batch size: 16
  Steps per epoch: 16
  Num epochs: 4
  Total steps: 64

Training args configured


In [ ]:
# Initialize the SFT Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=collate_fn,
    peft_config=None,
)

print("Trainer initialized")
print(f"  Train dataset: {len(trainer.train_dataset)} samples")
print(f"  Eval dataset: {len(trainer.eval_dataset)} samples")

Trainer initialized
  Train dataset: 264 samples
  Eval dataset: 20 samples


In [ ]:
# Start training
print("=" * 60)
print("STARTING TRAINING")
print("=" * 60)
print(f"Model: {MODEL_ID}")
print(f"Dataset: {DATASET_ID}")
print(f"Output: {OUTPUT_DIR}")
print(f"WandB: {WANDB_PROJECT}/{WANDB_RUN_NAME}")
print("=" * 60)

trainer.train()

print("\nTraining completed!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


STARTING TRAINING
Model: google/medgemma-1.5-4b-it
Dataset: Alfaxad/MedToolCalling
Output: medgemma-4b-tool-calling
WandB: MedGemmaAgent/Sara-1.5-4B-it


Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.


Step,Training Loss,Validation Loss
8,0.417919,0.099870
16,0.060760,0.041333
24,0.043044,0.027090
32,0.026334,0.016129
40,0.016686,0.015181
48,0.024874,0.014779
56,0.014584,0.014088
64,0.011958,0.013883



Training completed!


In [ ]:
# Save the final model
print("Saving model...")

# Save LoRA adapter
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to: {OUTPUT_DIR}")

# Log final metrics
final_metrics = trainer.state.log_history[-1] if trainer.state.log_history else {}
wandb.log({"final_metrics": final_metrics})

# List saved files
import os
saved_files = os.listdir(OUTPUT_DIR)
print(f"\nSaved files:")
for f in saved_files:
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / 1e6
    print(f"  {f}: {size:.2f} MB")

Saving model...
Model saved to: medgemma-4b-tool-calling

Saved files:
  tokenizer_config.json: 0.00 MB
  checkpoint-68: 0.00 MB
  checkpoint-32: 0.00 MB
  adapter_config.json: 0.00 MB
  training_args.bin: 0.01 MB
  checkpoint-16: 0.00 MB
  checkpoint-64: 0.00 MB
  checkpoint-48: 0.00 MB
  README.md: 0.00 MB
  tokenizer.json: 33.38 MB
  chat_template.jinja: 0.00 MB
  adapter_model.safetensors: 131.25 MB


In [ ]:
# Merge LoRA adapter with base model for standalone deployment
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("Loading base model for merging...")

# Load base model in full precision for merging
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

# Load and merge LoRA adapter
print("Merging LoRA adapter...")
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
model = model.merge_and_unload()

# Save merged model
MERGED_DIR = "medgemma-4b-tool-calling-merged"
print(f"Saving merged model to {MERGED_DIR}...")
model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print("Merge complete!")

# List saved files
import os
for f in os.listdir(MERGED_DIR):
    size = os.path.getsize(os.path.join(MERGED_DIR, f)) / 1e9
    print(f"  {f}: {size:.2f} GB")

Loading base model for merging...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

Merging LoRA adapter...
Saving merged model to medgemma-4b-tool-calling-merged...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Merge complete!
  tokenizer_config.json: 0.00 GB
  generation_config.json: 0.00 GB
  model.safetensors: 8.60 GB
  config.json: 0.00 GB
  tokenizer.json: 0.03 GB
  chat_template.jinja: 0.00 GB


In [ ]:
# Test inference with the merged fine-tuned model
print("Loading merged model for inference...")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load merged model
merged_model = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
merged_tokenizer = AutoTokenizer.from_pretrained(MERGED_DIR)

print(f"Model loaded from: {MERGED_DIR}")

# Get a sample from eval set
test_sample = eval_dataset[0]
test_messages = test_sample['messages']

# Use only the first user message (system prompt + question) for inference
# This simulates real usage where we give the task and expect the model to respond
first_turn = [test_messages[0]]

# Apply chat template with generation prompt
input_text = merged_tokenizer.apply_chat_template(
    first_turn,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize
inputs = merged_tokenizer(input_text, return_tensors="pt").to(merged_model.device)
input_len = inputs["input_ids"].shape[-1]

print(f"Input tokens: {input_len}")
print(f"Sample ID: {test_sample['id']}")

# Generate
print("\nGenerating response...")
with torch.inference_mode():
    outputs = merged_model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=merged_tokenizer.pad_token_id,
        eos_token_id=merged_tokenizer.eos_token_id,
    )

# Decode only the generated part
generated_tokens = outputs[0][input_len:]
generated_text = merged_tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n" + "=" * 60)
print("GENERATED OUTPUT:")
print("=" * 60)
print(generated_text)

print("\n" + "=" * 60)
print("EXPECTED OUTPUT:")
print("=" * 60)
print(test_messages[1]['content'])

# Check if they match
match = generated_text.strip() == test_messages[1]['content'].strip()
print("\n" + "=" * 60)
print(f"EXACT MATCH: {match}")
print("=" * 60)

Loading merged model for inference...


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

Model loaded from: medgemma-4b-tool-calling-merged
Input tokens: 3339
Sample ID: task3_5

Generating response...

GENERATED OUTPUT:
POST http://localhost:8080/fhir/Observation
{
  "resourceType": "Observation",
  "category": [
    {
      "coding": [
        {
          "system": "http://hl7.org/fhir/observation-category",
          "code": "vital-signs",
          "display": "Vital Signs"
        }
      ]
    }
  ],
  "code": {
    "text": "BP"
  },
  "effectiveDateTime": "2023-11-13T10:15:00+00:00",
  "status": "final",
  "valueString": "118/77 mmHg",
  "subject": {
    "reference": "Patient/S1521703"
  }
}

EXPECTED OUTPUT:
POST http://localhost:8080/fhir/Observation
{
  "resourceType": "Observation",
  "category": [
    {
      "coding": [
        {
          "system": "http://hl7.org/fhir/observation-category",
          "code": "vital-signs",
          "display": "Vital Signs"
        }
      ]
    }
  ],
  "code": {
    "text": "BP"
  },
  "effectiveDateTime": "2023-11-13T10:15

In [ ]:
# Finish the wandb run
wandb.finish()

print("WandB run finished")
print(f"View results at: https://wandb.ai/{WANDB_PROJECT}")

WandB run finished
View results at: https://wandb.ai/MedGemmaAgent


In [ ]:
# Push merged model to Hugging Face Hub (private repo)
from huggingface_hub import HfApi

HUB_MODEL_ID = ""

print(f"Uploading merged model to {HUB_MODEL_ID} (private)...")

# Push merged model
merged_model.push_to_hub(
    HUB_MODEL_ID,
    token=True,
    private=True,
)

# Push tokenizer
merged_tokenizer.push_to_hub(
    HUB_MODEL_ID,
    token=True,
    private=True,
)

print(f"Model pushed to: https://huggingface.co/{HUB_MODEL_ID}")
print("Repository is set to PRIVATE")

Uploading merged model to Alfaxad/Sara-1.5-4B-it (private)...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...k5vbyxu/model.safetensors:   0%|          |  112kB / 8.60GB            

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp8h6a20ns/tokenizer.json:  23%|##2       | 7.61MB / 33.4MB            

Model pushed to: https://huggingface.co/Alfaxad/Sara-1.5-4B-it
Repository is set to PRIVATE


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Zip the folder
import shutil

source_folder = "/content/medgemma-4b-tool-calling-merged"
output_zip = "/content/medgemma-4b-tool-calling-merged"  # .zip will be added automatically

print("Zipping folder...")
shutil.make_archive(output_zip, 'zip', source_folder)
print(f"Created: {output_zip}.zip")

# Copy to Google Drive
import shutil

destination = "/content/drive/MyDrive/medgemma-4b-tool-calling-merged.zip"

print("Copying to Google Drive...")
shutil.copy(f"{output_zip}.zip", destination)
print(f"Done! File saved to: {destination}")

Mounted at /content/drive
Zipping folder...
Created: /content/medgemma-4b-tool-calling-merged.zip
Copying to Google Drive...
Done! File saved to: /content/drive/MyDrive/medgemma-4b-tool-calling-merged.zip
